In [4]:
import pandas as pd
from collections import defaultdict
from itertools import combinations

sequences = [
    [['a'], ['a', 'b', 'c'], ['a', 'c'], ['d', 'e']],
    [['a', 'b'], ['c'], ['b', 'c', 'd']],
    [['b'], ['c', 'd', 'e']],
    [['a', 'c'], ['b', 'c'], ['a', 'b', 'c'], ['c', 'd', 'e']],
    [['a'], ['d'], ['c'], ['b'], ['e']]
]

for i, seq in enumerate(sequences):
    print(f"Séquence {i}: {seq}")

Séquence 0: [['a'], ['a', 'b', 'c'], ['a', 'c'], ['d', 'e']]
Séquence 1: [['a', 'b'], ['c'], ['b', 'c', 'd']]
Séquence 2: [['b'], ['c', 'd', 'e']]
Séquence 3: [['a', 'c'], ['b', 'c'], ['a', 'b', 'c'], ['c', 'd', 'e']]
Séquence 4: [['a'], ['d'], ['c'], ['b'], ['e']]


## 1. Algorithme GSP (Generalized Sequential Patterns)

### Principe
GSP est une extension d'Apriori aux données séquentielles. Il fonctionne de manière itérative :
1.  **Trouver les séquences fréquentes de longueur 1 :** On scanne la base de données pour trouver tous les articles uniques qui respectent le `min_support`.

2.  **Itérer pour k > 1 :**

    a. **Génération de Candidats (Join Step) :** À partir des séquences fréquentes de longueur `k-1`, on génère des séquences candidates de longueur `k`.

    
    b. **Élagage (Prune Step) :** On applique le **principe d'Apriori adapté aux séquences** : si une séquence est fréquente, alors toutes ses sous-séquences doivent l'être aussi. On élimine les candidats dont une sous-séquence est rare.


    c. **Comptage du Support :** On scanne la base de données pour compter le support des candidats restants et on ne garde que ceux qui dépassent `min_support`.

    
4.  **Répéter** jusqu'à ce qu'aucun nouveau candidat ne puisse être généré.

In [5]:
#Vérifier si 'sub' est une sous-séquence de 'main'.
def is_subsequence(sub, main):
    it = iter(main)
    return all(any(c in s for s in it) for c in sub)

#Calculer le support d'une séquence candidate.
def calculate_support(sequences, candidate):
    count = 0
    for seq in sequences:
        flat_seq = [item for sublist in seq for item in sublist]
        if is_subsequence(candidate, flat_seq):
            count += 1
    return count / len(sequences)

def generate_candidates(freq_sequences, k):
    candidates = set()
    for seq1 in freq_sequences:
        for seq2 in freq_sequences:
            if seq1[:-1] == seq2[:-1] and seq1[-1] < seq2[-1]:
                new_candidate = seq1 + (seq2[-1],)
                candidates.add(new_candidate)
    return list(candidates)

#Suppression des candidats non frequents
def prune(candidates, prev_freq_sequences, k):
    pruned_candidates = []
    for candidate in candidates:
        is_valid = True
        # Générer toutes les sous-séquences de longueur k-1
        for sub_seq in combinations(candidate, k - 1):
            if sub_seq not in prev_freq_sequences:
                is_valid = False
                break
        if is_valid:
            pruned_candidates.append(candidate)
    return pruned_candidates

def gsp(sequences, min_support):
    # Étape 1: Trouver les séquences fréquentes de longueur 1 (L1)
    item_counts = defaultdict(int)
    for seq in sequences:
        for item in set(item for sublist in seq for item in sublist):
            item_counts[item] += 1
    
    L1 = { (item,) for item, count in item_counts.items() if count / len(sequences) >= min_support }
    
    frequent_sequences = {1: L1}
    k = 2
    
    while True:
        # Générer des candidats de longueur k
        candidates = generate_candidates(list(frequent_sequences[k-1]), k)
        pruned_candidates = prune(candidates, frequent_sequences[k-1], k)
        # Compter le support des candidats
        Lk = set()
        for candidate in pruned_candidates:
            support = calculate_support(sequences, candidate)
            if support >= min_support:
                Lk.add(candidate)
        
        if not Lk:
            break
            
        frequent_sequences[k] = Lk
        k += 1
        
    return frequent_sequences

min_sup_gsp = 0.6
result_gsp = gsp(sequences, min_support=min_sup_gsp)

print(f"--- RÉSULTATS GSP (min_support = {min_sup_gsp}) ---\n")
for length, seqs in result_gsp.items():
    print(f"Séquences fréquentes de longueur {length}:")
    for seq in sorted(list(seqs)):
        print(f"  - {seq}")

--- RÉSULTATS GSP (min_support = 0.6) ---

Séquences fréquentes de longueur 1:
  - ('a',)
  - ('b',)
  - ('c',)
  - ('d',)
  - ('e',)
Séquences fréquentes de longueur 2:
  - ('a', 'b')
  - ('a', 'c')
  - ('a', 'd')
  - ('a', 'e')
  - ('b', 'c')
  - ('b', 'd')
  - ('b', 'e')
  - ('c', 'd')
  - ('c', 'e')
  - ('d', 'e')
Séquences fréquentes de longueur 3:
  - ('a', 'b', 'c')
  - ('a', 'b', 'd')
  - ('a', 'b', 'e')
  - ('a', 'c', 'd')
  - ('a', 'c', 'e')
  - ('a', 'd', 'e')
  - ('b', 'c', 'd')
  - ('b', 'c', 'e')
  - ('b', 'd', 'e')
  - ('c', 'd', 'e')
Séquences fréquentes de longueur 4:
  - ('a', 'b', 'c', 'd')
  - ('b', 'c', 'd', 'e')


## 2. Algorithme SPADE (Sequential PAttern Discovery using Equivalence classes)

### Principe
SPADE adopte une approche radicalement différente de GSP pour éviter les scans répétés de la base de données, qui sont très coûteux. Il utilise une **représentation verticale des données**.

1.  **Création de la base de données verticale :** Au lieu de `Séquence -> [Événements]`, SPADE crée une table inversée : `Item -> [(ID_Séquence, ID_Événement)]`. Cette table, appelée **id-list**, contient pour chaque article la liste de tous les moments où il apparaît.
  
2.  **Décomposition du problème :** L'algorithme décompose le problème de recherche en "classes d'équivalence" basées sur les préfixes. Par exemple, il trouve d'abord toutes les séquences qui commencent par `{a}`, puis toutes celles qui commencent par `{b}`, etc.
  
4.  **Joindre les id-lists :** Pour trouver le support d'une séquence comme `<{a}, {b}>`, SPADE n'a qu'à joindre les id-lists de `{a}` et de `{b}` pour voir quand `{b}` apparaît *après* `{a}` dans les mêmes séquences. Cette opération est beaucoup plus rapide qu'un scan complet de la base de données.

In [6]:
# Creation de l'id-list
def create_vertical_database(sequences):
    db = defaultdict(list)
    for sid, seq in enumerate(sequences):
        for eid, itemset in enumerate(seq):
            for item in itemset:
                db[item].append((sid, eid))
    return db

def spade_recursive(prefix, id_list, min_support_count, results):
    # Trouver les items fréquents qui peuvent étendre le préfixe
    frequent_items = []
    
    # Créer une base temporaire pour les items qui suivent le préfixe
    temp_db = defaultdict(list)
    for sid, eid in id_list:
        # Regarder les événements après (sid, eid) dans la séquence originale
        for next_eid in range(eid + 1, len(sequences[sid])):
            for item in sequences[sid][next_eid]:
                temp_db[item].append((sid, next_eid))
                
    # Filtrer les items fréquents
    for item, new_id_list in temp_db.items():
        support_count = len(set(sid for sid, eid in new_id_list))
        if support_count >= min_support_count:
            frequent_items.append((item, new_id_list))

    # Pour chaque item fréquent, l'ajouter au préfixe et continuer récursivement
    for item, new_id_list in frequent_items:
        new_prefix = prefix + (item,)
        results.append(new_prefix)
        spade_recursive(new_prefix, new_id_list, min_support_count, results)

def spade_main(sequences, min_support):
    min_support_count = min_support * len(sequences)
    vertical_db = create_vertical_database(sequences)
    
    frequent_sequences = []
    
    #Recherche pour chaque item de longueur 1
    initial_items = sorted(vertical_db.keys())
    
    for item in initial_items:
        id_list = vertical_db[item]
        support_count = len(set(sid for sid, eid in id_list))
        
        if support_count >= min_support_count:
            prefix = (item,)
            frequent_sequences.append(prefix)
            spade_recursive(prefix, id_list, min_support_count, frequent_sequences)
            
    return frequent_sequences

min_sup_spade = 0.6 
result_spade = spade_main(sequences, min_support=min_sup_spade)

print(f"\n--- RÉSULTATS SPADE (min_support = {min_sup_spade}) ---\n")
results_by_length = defaultdict(list)
for seq in result_spade:
    results_by_length[len(seq)].append(seq)

for length in sorted(results_by_length.keys()):
    print(f"Séquences fréquentes de longueur {length}:")
    for seq in sorted(results_by_length[length]):
        print(f"  - {seq}")


--- RÉSULTATS SPADE (min_support = 0.6) ---

Séquences fréquentes de longueur 1:
  - ('a',)
  - ('b',)
  - ('c',)
  - ('d',)
  - ('e',)
Séquences fréquentes de longueur 2:
  - ('a', 'b')
  - ('a', 'c')
  - ('a', 'd')
  - ('a', 'e')
  - ('b', 'c')
  - ('b', 'd')
  - ('b', 'e')
  - ('c', 'b')
  - ('c', 'c')
  - ('c', 'd')
  - ('c', 'e')
Séquences fréquentes de longueur 3:
  - ('a', 'b', 'e')
  - ('a', 'c', 'b')
  - ('a', 'c', 'c')
  - ('a', 'c', 'd')
  - ('a', 'c', 'e')
  - ('b', 'c', 'd')
